In [1]:
!ls


 data_perterbers			    ParlAI
'Exploratory Data Analysis.ipynb'	    raw_data
'HAP Filter.ipynb'			    ResponsibleNLP
 logs					    results
 nltk_data				    style-transfer-paraphrase
'PANDA Copy1.ipynb'			    test_trainer
'PANDA Data Perturber on HAP-Copy1.ipynb'   value
'PANDA Data Perturber on HAP.ipynb'	   'VALUE implementation on HAP.ipynb'


In [7]:
!git clone https://github.com/SALT-NLP/value.git

fatal: destination path 'value' already exists and is not an empty directory.


In [8]:
!git clone https://github.com/martiansideofthemoon/style-transfer-paraphrase.git

fatal: destination path 'style-transfer-paraphrase' already exists and is not an empty directory.


In [10]:
!cd style-transfer-paraphrase
!ls

 data_perterbers
'Exploratory Data Analysis.ipynb'
'HAP Filter.ipynb'
 nltk_data
 raw_data
 style-transfer-paraphrase
 value
'VALUE and PANDA implementation on HAP.ipynb'


In [15]:
!conda create --name value python=3.7
!y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /u/camilleh/.conda/envs/value

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1u             |       h7f8727e_0         3.8 MB
    pip-22.3.1                 |   py37h06a4308_0         2.7 MB
    python-3.7.13              |       h12debd9_0        53.5 MB
    setuptools-65.6.3          |   py37h06a4308_0         1.4 MB
    wheel-0.38.4               |   py37h06a4308_0          57 KB
    ------------------------------------------------------------
                                           Total:        61.4 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkg

In [16]:
y

NameError: name 'y' is not defined

In [ ]:
conda activate value


In [2]:
pip install -r 'value/requirements.txt'

  Using cached nltk-3.5.zip (1.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.19.2-cp37-cp37m-manylinux2010_x86_64.whl (14.5 MB)
  Using cached pandas-1.1.3-cp37-cp37m-manylinux1_x86_64.whl (9.5 MB)
  Using cached tqdm-4.50.2-py2.py3-none-any.whl (70 kB)
  Using cached neuralcoref-4.0-cp37-cp37m-manylinux1_x86_64.whl (286 kB)
  Using cached lemminflect-0.2.2-py3-none-any.whl (769 kB)
  Using cached spacy-2.1.0-cp37-cp37m-manylinux1_x86_64.whl (27.7 MB)
  Using cached inflect-5.5.2-py3-none-any.whl (33 kB)
  Using cached torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl (776.8 MB)
  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Using cached datasets-2.1.0-py3-none-any.whl (325 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.7 MB/s eta 0:00:00a 0:00:01
  Using cached joblib-1.3.1-py3-none-any.whl (301 kB)
  Using cached regex-2023.6.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (755 kB)
  Using cached pytz-2023.3-py2.py3-non

In [9]:
!python 'style-transfer-paraphrase/style_paraphrase/inference_utils.py'


Traceback (most recent call last):
  File "style-transfer-paraphrase/style_paraphrase/inference_utils.py", line 8, in <module>
    from style_paraphrase.dataset_config import DATASET_CONFIG, BASE_CONFIG
ModuleNotFoundError: No module named 'style_paraphrase'


In [6]:
pip install -r requirements.txt


ModuleNotFoundError: No module named 'style_paraphrase'

In [11]:
!python -m spacy download en_core_web_sm

/u/camilleh/.conda/envs/t4j/bin/python: No module named spacy


In [2]:
import pandas as pd
import sklearn 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from nltk.corpus import stopwords


from sklearn.feature_extraction.text import CountVectorizer
import nltk
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


In [3]:
toxic_200_train = pd.read_json('raw_data/commercial-use-allowed/en/toxic200_en/train.json')
t200_train = pd.json_normalize(toxic_200_train['data'])


In [4]:

paraphraser = GPT2Generator('pretrained_style_transfer/models/paraphraser_gpt2_large/', upper_length="same_5")
paraphraser.modify_p(top_p=0.6)
sae_to_aave = GPT2Generator('pretrained_style_transfer/models/cds_models/aae', upper_length="same_5")
sae_to_aave.modify_p(top_p=0.6)

df = t200_train # FILL THIS PATH IN
converted = []
batch_size = 32
for i in range(int(len(df)/batch_size)+1):
    sub_df = df.iloc[batch_size*(i):batch_size*(i+1)].copy()
    for col in df.columns:
        if (('text' in col) or ('question' in col)) and ('parse' not in col):
            sub_df = sub_df[[type(c)==str for c in sub_df[col].values]].copy()
            consider = sub_df[col].values
            para, prob = paraphraser.generate_batch(consider)
            aave, prob = sae_to_aave.generate_batch(para)
            sub_df[col+'-glue'] = consider
            sub_df[col] = aave
            converted.append(sub_df)
            
converted_df = pd.concat(converted)
converted_df.to_csv()

NameError: name 'GPT2Generator' is not defined